In [2]:
%%bash
doormat login -f && eval $(doormat aws -a aws_jose.merchan_test export)
# Function to retrieve certificate from Secrets Manager
retrieve_cert() {
    local secret_name="$1"
    local output_file="$2"
    local description="$3"
    
    for i in {1..30}; do
        if aws secretsmanager describe-secret --secret-id "$secret_name" --region eu-west-2 >/dev/null 2>&1; then
            aws secretsmanager get-secret-value --secret-id "$secret_name" --region eu-west-2 --query 'SecretString' --output text > "$output_file" 2>/dev/null
            if [ -s "$output_file" ]; then
                echo "✅ Retrieved $description"
                return 0
            fi
        fi
        echo "Waiting for $description... ($i/30)"
        sleep 5
    done
    echo "❌ Failed to retrieve $description"
    return 1
}

# Retrieve certificates
retrieve_cert "nomad-ca-initSecret-mjlf" "nomad-agent-ca.pem" "CA certificate"
retrieve_cert "nomad-client-cert-initSecret-mjlf" "dc1-client-nomad.pem" "client certificate"  
retrieve_cert "nomad-client-key-initSecret-mjlf" "dc1-client-nomad-key.pem" "client private key"

# Set proper permissions
chmod 0644 nomad-agent-ca.pem
chmod 0644 dc1-client-nomad.pem
chmod 0600 dc1-client-nomad-key.pem

time="2025-11-19T16:09:31+01:00" level=warning msg="new doormat cli recommended: v4.6.0 run `doormat update` to update!"
time="2025-11-19T16:09:31+01:00" level=info msg="logging into doormat..."
time="2025-11-19T16:09:35+01:00" level=info msg="successfully logged into doormat!"


✅ Retrieved CA certificate
✅ Retrieved client certificate
✅ Retrieved client private key


In [ ]:
%%bash
mkdir -p nomad
mkdir -p nomad/data

mv dc1-client-nomad.pem nomad/dc1-client-nomad.pem
mv dc1-client-nomad-key.pem nomad/dc1-client-nomad-key.pem
mv nomad-agent-ca.pem nomad/nomad-agent-ca.pem

In [21]:
%%bash
# Create Nomad client configuration

echo "Creating Nomad client configuration..."

cat <<EOF > nomad/client.hcl
# Data directory for Nomad client
data_dir = "/Users/jose/Demo/INDITEX_NOMAD_VAULT/nomad/data"


region     = "dc1"
datacenter = "dc1"

client {
  enabled = true

  # Server address (use RPC port 4647 for TLS)
  servers = ["13.135.51.29:4647"]

  # Node class for targeting
  node_class = "amazon-linux-client"

  options {
    "driver.raw_exec.enable" = "1"
    "driver.docker.enable"   = "0"
    "driver.qemu.enable"     = "1"
    "driver.podman.enable"   = "1" 
  }
}

plugin "nomad-driver-podman" {
  config {
    allow_privileged = true
    volumes {
      enabled = true
    }
    gc {
      image = true
      container = true
    }
  }
}

plugin "qemu" {
  config {
    image_paths = ["/opt/nomad/qemu"]
    args_allowlist = [
      "-device",
      "-drive",
      "-netdev",
      "-cdrom",
      "-boot",
      "-m",
      "-smp",
      "-enable-kvm",
      "-cpu",
      "-machine"
    ]
  }
}

# TLS configuration for client
tls {
  http = true
  rpc  = true

  ca_file   = "/Users/jose/Demo/INDITEX_NOMAD_VAULT/1_create_clusters/nomad/nomad-agent-ca.pem"
  cert_file = "/Users/jose/Demo/INDITEX_NOMAD_VAULT/1_create_clusters/nomad/dc1-client-nomad.pem"
  key_file  = "/Users/jose/Demo/INDITEX_NOMAD_VAULT/1_create_clusters/nomad/dc1-client-nomad-key.pem"

  verify_server_hostname = true
  verify_https_client    = false
}

# ACL configuration for client
acl {
  enabled    = true
}
EOF

Creating Nomad client configuration...


In [ ]:
%%bash
nomad agent  -config=/Users/jose/Demo/INDITEX_NOMAD_VAULT/1_create_clusters/nomad/client.hcl